In [1]:
import gurobipy
import pandas as pd
from IORFA import *
import numpy as np
import time

In [2]:
from sklearn.datasets import make_regression

# Generate a regression dataset with 100 samples, 1 feature, and 0.1 noise
X, y, coef = make_regression(n_samples=1000, n_features=10, noise=0, random_state= 0, n_informative=1, coef = True)

scales = np.max(X, axis=0)
scales[scales == 0] = 1
X = X/scales

In [3]:
train_size = 800
X_train = X[:train_size]
y_train = y[:train_size]

X_test = X[train_size:]
y_test = y[train_size:]

In [4]:
coef

array([0.        , 0.        , 0.        , 0.        , 9.56666472,
       0.        , 0.        , 0.        , 0.        , 0.        ])

In [5]:
ORFA = optimalDecisionTreeClassifier(max_depth=2,output=False, warmstart = True)

In [6]:
t0 = time.time()
ORFA.fit(X_train, y_train, scaled = True)
t1 = time.time()
total = t1-t0
print(total)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-05-19


/Users/ryanlucas/Downloads/An-Optimal-RuleFit-Algorithm-main/IORFA/IORFA.py:204: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode = stats.mode(y)[0][0]


0.8127171993255615


In [7]:
sum(ORFA._b.values())

0.2957978581178524

In [8]:
def find_rules_from_node_x(x, odt):

    # Find the active splits (those that are actually used) at the first node
    active_splits_at_node_x =  {i: odt._a[i] for i in list(odt._a.keys()) if (i[1] == x and odt._a[i] == 1)}

    # Store the splitting variable and the threshold together
    for key in list(active_splits_at_node_x.keys()):
        active_splits_at_node_x[(key[0], odt._b[x])] = active_splits_at_node_x[key]
        del active_splits_at_node_x[key]

    # Make rules out of them
    rules_from_node_x = {(f"np.array(X[:, {key[0]}]) <= " + str(key[1]), 
                        f"np.array(X[:, {key[0]}]) > " + str(key[1]) )           
                        for key in active_splits_at_node_x.keys()}
    
    return rules_from_node_x

def chain_rules(odt):
    
    rules_N1 = list(find_rules_from_node_x(1, odt))[0]
    rules_N2 = list(find_rules_from_node_x(2, odt))[0]
    rules_N3 = list(find_rules_from_node_x(3, odt))[0]

    path1 = "(" + rules_N1[0] + ")" + " * " + "(" + (rules_N2[0]) + ")"
    path2 = "(" +(rules_N1[0])+ ")" + " * " + "(" + (rules_N2[1]) + ")"
    path3 = "(" +(rules_N1[1]) + ")" + " * " +  "(" +(rules_N3[0]) + ")"
    path4 = "(" +(rules_N1[1]) + ")" + " * " + "(" +(rules_N3[1]) + ")"


    return [path1, path2, path3, path4]


def add_rules_to_df(rules, X):

    X = np.matrix(X)

    for rule in rules:
        rule_ind = []

        for k in range(X.shape[0]):
            if k in np.where(np.array(eval(rule)).flatten())[0]:
                rule_ind.append(1)
            else:
                rule_ind.append(0)
    
        X = np.hstack((X, np.array(rule_ind).reshape(-1,1)))
    
    return X

def IORFA(X, odt):
    rules = chain_rules(odt)

    betas = [odt.beta[i] for i in list(odt.beta.keys())]
    gammas = [odt.gamma[i] for i in list(odt.gamma.keys())]
    coefficients = betas + gammas
    rule_df = np.matrix(add_rules_to_df(rules, X))

    preds = []
    for i in range(X.shape[0]):
        print(rule_df.shape)

        y_i = sum(rule_df[i, j]*coefficients[j] for j in range(len(coefficients)))
        preds.append(y_i)
    
    return preds

In [9]:
y_train_pred = IORFA(X_train, ORFA)

(800, 14)
(800, 14)
(800, 14)
(800, 14)
(800, 14)
(800, 14)
(800, 14)
(800, 14)
(800, 14)
(800, 14)
(800, 14)
(800, 14)
(800, 14)
(800, 14)
(800, 14)
(800, 14)
(800, 14)
(800, 14)
(800, 14)
(800, 14)
(800, 14)
(800, 14)
(800, 14)
(800, 14)
(800, 14)
(800, 14)
(800, 14)
(800, 14)
(800, 14)
(800, 14)
(800, 14)
(800, 14)
(800, 14)
(800, 14)
(800, 14)
(800, 14)
(800, 14)
(800, 14)
(800, 14)
(800, 14)
(800, 14)
(800, 14)
(800, 14)
(800, 14)
(800, 14)
(800, 14)
(800, 14)
(800, 14)
(800, 14)
(800, 14)
(800, 14)
(800, 14)
(800, 14)
(800, 14)
(800, 14)
(800, 14)
(800, 14)
(800, 14)
(800, 14)
(800, 14)
(800, 14)
(800, 14)
(800, 14)
(800, 14)
(800, 14)
(800, 14)
(800, 14)
(800, 14)
(800, 14)
(800, 14)
(800, 14)
(800, 14)
(800, 14)
(800, 14)
(800, 14)
(800, 14)
(800, 14)
(800, 14)
(800, 14)
(800, 14)
(800, 14)
(800, 14)
(800, 14)
(800, 14)
(800, 14)
(800, 14)
(800, 14)
(800, 14)
(800, 14)
(800, 14)
(800, 14)
(800, 14)
(800, 14)
(800, 14)
(800, 14)
(800, 14)
(800, 14)
(800, 14)
(800, 14)
(800, 14)


In [10]:
from sklearn.metrics import r2_score
r2_score(y_train_pred, y_train)

1.0

In [11]:
y_test_pred = IORFA(X_test, ORFA)

(200, 14)
(200, 14)
(200, 14)
(200, 14)
(200, 14)
(200, 14)
(200, 14)
(200, 14)
(200, 14)
(200, 14)
(200, 14)
(200, 14)
(200, 14)
(200, 14)
(200, 14)
(200, 14)
(200, 14)
(200, 14)
(200, 14)
(200, 14)
(200, 14)
(200, 14)
(200, 14)
(200, 14)
(200, 14)
(200, 14)
(200, 14)
(200, 14)
(200, 14)
(200, 14)
(200, 14)
(200, 14)
(200, 14)
(200, 14)
(200, 14)
(200, 14)
(200, 14)
(200, 14)
(200, 14)
(200, 14)
(200, 14)
(200, 14)
(200, 14)
(200, 14)
(200, 14)
(200, 14)
(200, 14)
(200, 14)
(200, 14)
(200, 14)
(200, 14)
(200, 14)
(200, 14)
(200, 14)
(200, 14)
(200, 14)
(200, 14)
(200, 14)
(200, 14)
(200, 14)
(200, 14)
(200, 14)
(200, 14)
(200, 14)
(200, 14)
(200, 14)
(200, 14)
(200, 14)
(200, 14)
(200, 14)
(200, 14)
(200, 14)
(200, 14)
(200, 14)
(200, 14)
(200, 14)
(200, 14)
(200, 14)
(200, 14)
(200, 14)
(200, 14)
(200, 14)
(200, 14)
(200, 14)
(200, 14)
(200, 14)
(200, 14)
(200, 14)
(200, 14)
(200, 14)
(200, 14)
(200, 14)
(200, 14)
(200, 14)
(200, 14)
(200, 14)
(200, 14)
(200, 14)
(200, 14)
(200, 14)


In [12]:
import matplotlib.pyplot as plt

r2_score(y_test_pred, y_test)

1.0